**If the dataprocessing is done in parallel, this below block can be run**

In [1]:
import pandas as pd
import os 

path = "/kaggle/input/traindata/finaldata/finaldata"
num = len(os.listdir(path))
output_file = "final_test22.csv"

dfs = []

for i in range(num):
    file = f"{path}/f_{i+1}.csv"
    df = pd.read_csv(file)
    dfs.append(df)
    


    
combined_df = pd.concat(dfs, ignore_index=True)

print(len(combined_df))
combined_df.to_csv(output_file, index=False)

print(f"Combined CSV file saved as {output_file}")

131187
Combined CSV file saved as final_test22.csv


# **********FINAL RUN**

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import VisualBertModel, VisualBertConfig, BertTokenizer, BlipProcessor, BlipForConditionalGeneration
from transformers import DistilBertTokenizer, DistilBertModel
from torchvision import transforms, models
from PIL import Image
from io import BytesIO
import requests
import pandas as pd
from tqdm import tqdm

In [3]:
class DistilBERTRegressor(nn.Module):
    def __init__(self, output_dim=1):
        super(DistilBERTRegressor, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.fc = nn.Linear(self.distilbert.config.hidden_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, output_dim)
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        last_hidden_state = outputs[0]
        pooled_output = torch.mean(last_hidden_state, 1)
        x = pooled_output
        x = self.relu(self.fc(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

In [4]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

model_h = DistilBERTRegressor(output_dim=1)
model_w = DistilBERTRegressor(output_dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.device_count() > 1:
    model_h = torch.nn.DataParallel(model_h)
    model_w = torch.nn.DataParallel(model_w)
    
model_h = model_h.to(device)
model_w = model_w.to(device)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
d=pd.read_csv("/kaggle/working/final_test22.csv")
d['text'] = d['text'].str.replace("a close up of a", "", regex=False)
print(d.iloc[0]["entity_name"])
df=d

height


In [6]:
weights1="/kaggle/input/traindata/distilbert_model_imageh.pth"
checkpoint1 = torch.load(weights1, map_location=device)
model_h.load_state_dict(checkpoint1['model_state_dict'])

/tmp/ipykernel_36/910145131.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint1 = torch.load(weights1, map_location=device)


<All keys matched successfully>

In [7]:
weights2="/kaggle/input/traindata/distilbert_model_imagew.pth"
checkpoint2 = torch.load(weights1, map_location=device)
model_w.load_state_dict(checkpoint2['model_state_dict'])

/tmp/ipykernel_36/698221403.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint2 = torch.load(weights1, map_location=device)


<All keys matched successfully>

In [8]:
import numpy as np
print(len(df))

131187


In [9]:
first = ["height", "width", "depth"]
second = ["item_weight"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

df['prediction'] = np.nan

with torch.no_grad():

    for i in range(len(df)):
        input_text = df.iloc[i]["text"]
        entity_name = str(df.iloc[i]["entity_name"]).lower()

        if len(input_text) > 2 and (entity_name in first or entity_name in second):

            inputs = tokenizer.encode_plus( input_text, add_special_tokens=True, max_length=32, return_token_type_ids=False, padding='max_length', truncation=True, return_attention_mask=True, return_tensors='pt' )
            
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)

            if entity_name in first:
                outputs = model_h(input_ids, attention_mask)
                value = outputs.squeeze().item()  
                unit = "metre"
                final_value = f"{value:.2f} {unit}"

            elif entity_name in second:
                outputs = model_w(input_ids, attention_mask)
                value = outputs.squeeze().item()  
                unit = "ton"
                final_value = f"{value:.2f} {unit}"

        else:
            final_value = np.nan
            
        
        df.at[i, 'prediction'] = final_value
#         print(f"{i} of {len(df)}")

df = df[['index', 'prediction']]



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/tmp/ipykernel_36/3209459315.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.53 metre' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'prediction'] = final_value
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [10]:
df.to_csv("submission.csv", index=False)